In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import sys
import os

from pathlib import Path
from scipy.io import savemat
from scipy.stats import ttest_rel, ttest_1samp, pearsonr, f_oneway, zscore
from statsmodels.stats.multitest import multipletests
from statsmodels.formula.api import ols

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data
from dn_utils.plotting import aligned_imshow_cbar
from dn_utils.networks import zscore_network

%matplotlib inline
mpl.rcParams['font.size'] = 13

In [ ]:
# Behavioral modelling parameters
alpha_pdci_mle = np.load(os.path.join(
    path_jags, 
    'parameter_estimates/alpha_pdci_mle_3digits.npy'))
beta_pdci_mle = np.load(os.path.join(
    path_jags, 
    'parameter_estimates/beta_pdci_mle_3digits.npy'))

## Question 2
---
Comparison on the level of single connections

- Which connections significantly differ from 0 in reward condition?
- Which connections significantly differ from 0 in punishment condition?
- Which connections significantly differ between conditions?

In [ ]:
beta_mat_neqzero_rew = np.zeros((3, n_rois, n_rois))
beta_mat_neqzero_pun = np.zeros((3, n_rois, n_rois))
beta_mat_rew_minus_pun = np.zeros((3, n_rois, n_rois))

triu_idx = np.triu_indices(n_rois, 1)

for i in range(n_rois):
    for j in range(n_rois):
        if i != j:
            
            beta_vec_rew = beta_mats_agg_trans[:, meta['dim2'].index('rew'), 
                                             beta_idx, i, j]
            beta_vec_pun = beta_mats_agg_trans[:, meta['dim2'].index('pun'), 
                                             beta_idx, i, j]

            beta_mat_neqzero_rew[:2, i, j] = np.array(
                ttest_1samp(beta_vec_rew, popmean=0))
            beta_mat_neqzero_pun[:2, i, j] = np.array(
                ttest_1samp(beta_vec_pun, popmean=0))
            beta_mat_rew_minus_pun[:2, i, j] = np.array(
                ttest_1samp(beta_vec_rew - beta_vec_pun, popmean=0))
        
# Correct for multiple comparisons
beta_mat_neqzero_rew[2][triu_idx] = multipletests(
    beta_mat_neqzero_rew[1][triu_idx], alpha=0.05, method='fdr_bh')[1]
beta_mat_neqzero_pun[2][triu_idx] = multipletests(
    beta_mat_neqzero_pun[1][triu_idx], alpha=0.05, method='fdr_bh')[1]
beta_mat_rew_minus_pun[2][triu_idx] = multipletests(
    beta_mat_rew_minus_pun[1][triu_idx], alpha=0.05, method='fdr_bh')[1]

beta_mat_neqzero_rew[2] = beta_mat_neqzero_rew[2] + beta_mat_neqzero_rew[2].T 
beta_mat_neqzero_pun[2] = beta_mat_neqzero_pun[2] + beta_mat_neqzero_pun[2].T 
beta_mat_rew_minus_pun[2] = beta_mat_rew_minus_pun[2] \
                          + beta_mat_rew_minus_pun[2].T 

In [ ]:
for corr_mat, effect_name in zip(
    (beta_mat_neqzero_rew, beta_mat_neqzero_pun, beta_mat_rew_minus_pun), 
    ('rew != 0', 'pun != 0', 'rew != pun')
):

    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(
        #corr_mat[0] * (corr_mat[1] < 0.05),
        corr_mat[0],
        cmap='RdBu_r'
    )

    ax.set_xticks(np.arange(n_rois))
    ax.set_yticks(np.arange(n_rois))
    ax.set_xticklabels(df_custom['Regions'])
    ax.set_yticklabels(df_custom['Regions'])
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right',
             rotation_mode='anchor')

    # Colorbar
    im.set_clim([-max(np.abs(im.get_clim())), max(np.abs(im.get_clim()))])
    aligned_imshow_cbar(ax, im)

    # Add FDR corrected significance
    for i,j in zip(*np.nonzero(corr_mat[2] < 0.05)):
        if i != j:
            ax.text(j, i+0.1, '*', ha='center', va='center', color='w', fontSize=20)

    # Separate networks
    ax.axvline(6.5, color='k')
    ax.axhline(6.5, color='k')
    ax.set_title(effect_name)

## Question 3
---
Does change in connectivity between right and left VS and vmPFC correlates with difference in learning rates?

In [ ]:
idx_vmpfc = (df_custom['Regions'] == 'Ventromedial Prefrontal Cortex').argmax()
idx_vs_r = ((df_custom['Regions'] == 'Ventral striatum') \
          & (df_custom['Hemisphere'] == 'R')).argmax()
idx_vs_l = ((df_custom['Regions'] == 'Ventral striatum') \
          & (df_custom['Hemisphere'] == 'L')).argmax()


for roi_idx, roi_name in zip((idx_vs_l, idx_vs_r), ('lVS', 'rVS')):
    for con, con_name in enumerate(meta['dim2']):

        vec_beh = alpha_pdci_mle[ok_index, 0] - alpha_pdci_mle[ok_index, 1] 
        vec_net = beta_mats_agg_trans[:, con, beta_idx, idx_vmpfc, roi_idx] 

        ax = sns.jointplot(
            vec_beh, 
            vec_net, 
            kind='reg',
            xlim=[-0.6, 1],
            color='k', 
        )
        plt.xlabel(r'$\alpha_+ - \alpha_-$')
        plt.ylabel(r'$\beta_{VS – vmPFC}$')
        ax.fig.suptitle('(roi: {}, con: {}) r = {:.2f}; p-value = {:.5f}'.format(
            roi_name, con_name, *pearsonr(vec_beh, vec_net)))
        ax.fig.tight_layout()
        ax.fig.subplots_adjust(top=0.95) # Reduce plot to make room 